In [128]:
import requests
import glob
import sys
import csv
import os
import time
import pandas as pd
from tqdm import tqdm
import os
import sqlite3
# from sqlalchemy import create_engine, Table, MetaData, text
import numpy as np

In [35]:
pd.set_option("display.max_columns",100)

In [36]:
# jcode=pd.read_csv("Z:/都市構造データ/yahoopoi/jcode.csv",encoding="shift-jis",dtype=object)
JCODE = "47207"
# ベースとなるURL
API_KEY = "dj00aiZpPUlqUGhLTkRONXZDbiZzPWNvbnN1bWVyc2VjcmV0Jng9NDM-"  # ここにAPIキーを直接記述してください

#genreの台帳作成
genre=pd.read_csv("Y:/都市構造データ/yahoopoi/genre.csv",dtype=object)
genre["genre_name"]=genre["業種名1"]+"_"+genre["業種名2"]+"_"+genre["業種名3"]
genre_dict=dict(zip(genre.業種コード3,genre.genre_name))
#genreを選択
genre_code1=genre[genre["業種コード1"]=="01"]
genre_list=list(genre_code1["業種コード3"])

In [78]:
# データ取得用の関数
def fetch_data(params_02, total_num):
    base_url = "https://map.yahooapis.jp/search/local/V1/localSearch"
    max_return = 100  # APIの仕様では一回のリクエストにつき100件まで取得可能
    pages = (int(total_num) // int(max_return)) + 1  # 全件を取得するために必要なリクエスト回数を算定

    params_02['results'] = max_return  # 全件取得用のパラメータを設定

    Records = []  # 取得データを格納するための空リストを用意

    # 全件取得するためのループ処理
    for i in range(pages):
        i_startRecord = 1 + (i * int(max_return))
        params_02['start'] = i_startRecord
        response_02 = requests.get(base_url, params=params_02)

        # レスポンスのステータスが200＝正常取得だった場合の処理
        if response_02.status_code == 200:
            try:
                jsonData_02 = response_02.json()  # レスポンスをJSONデータとして格納する
            except ValueError:
                # print("エラー: レスポンスデータの解析処理に失敗しました。")
                continue
        else:
            # print("エラー:", response_02.status_code)
            continue

        # JSONデータ内の各要素をそのままリストに格納する
        for poi in jsonData_02.get('Feature', []):
            Records.append(poi)

        sys.stdout.flush()
        time.sleep(0.5)

    return Records

# フラット化
def flatten(d, parent_key='', sep='.'):
    items = []
    for k, v in d.items():
        #列名の生成
        new_key = parent_key + sep + k if parent_key else k
        # 辞書型項目のフラット化
        if isinstance(v, dict):
            items.extend(flatten(v, new_key, sep=sep).items())
        # リスト項目のフラット化
        elif isinstance(v, list):
            new_key_tmp = new_key
            for i, elm in enumerate(v):
                new_key = new_key_tmp + sep + str(i)
                # リストの中の辞書
                if isinstance(elm, dict):
                    items.extend(flatten(elm, new_key, sep=sep).items())
                # 単なるリスト
                else:
                    items.append((new_key, elm))
        # 値追加
        else:
            items.append((new_key, v))
    return dict(items)
def scraping_jcode(category_code, api_key, jcode):
    base_url = "https://map.yahooapis.jp/search/local/V1/localSearch"
    # URLパラメータ用の辞書を用意
    params_01 = {
        "appid": api_key,
        "results": 1,
        "output": "json",
        "gc": category_code
    }
    params_02 = {
        "appid": api_key,
        "sort": "kana",
        "output": "json",
        "gc": category_code,
        "detail": "full"  # すべての詳細項目を取得
    }

    params_01['ac'] = jcode
    params_02['ac'] = jcode


    # ヒット件数の確認用のリクエストを投げる処理
    try:
        response_01 = requests.get(base_url, params=params_01)
        jsonData_01 = response_01.json()
        total_num_each = jsonData_01["ResultInfo"]["Total"]
    except ValueError:
        pass
    
    # ヒット件数が0件以上かつ取得条件の3100件以内だった場合は取得処理を実行
    if total_num_each > 3100:
        print(f"データ取得上限の件数を超えているため、取得処理をスキップします。")
    elif total_num_each > 0:
        r = fetch_data(params_02, total_num_each)
        # フラット化
        dlist = [flatten(di) for di in r]
        df = pd.DataFrame.from_dict(dlist)


        col = ['Id', 'Gid', 'Name', 'Geometry.Coordinates', 'Property.Uid', 'Property.Yomi',
               'Property.Country.Code', 'Property.Country.Name', 'Property.Address',
               'Property.GovernmentCode', 'Property.Tel1', 'Property.Genre.0.Code',
               'Property.Genre.0.Name', 'Property.Building.0.Id', 'Property.Building.0.Name',
               'Property.Building.0.Floor', 'Property.Building.0.Area']



        rename_col = ['Id', 'Gid', 'Name', 'Geometry', 'Uid', 'Yomi', 'CountryCode',
                      'CountryName', 'Address', 'GovernmentCode', 'Tel', 'GenreCode',
                      'GenreName', 'BuildingId', 'BuildingName', 'BuildingFloor',
                      'BuildingArea']
        for c in col:
            if c in list(df.columns):
                pass
            else:
                df[c]=np.nan

        df_ = df[col].rename(columns=dict(zip(col, rename_col)))
        df_["X"] = df_["Geometry"].apply(lambda x: x.split(",")[0])
        df_["Y"] = df_["Geometry"].apply(lambda x: x.split(",")[1])

        # データを追加
        #df_.to_csv(output_name)
        return df_

In [79]:
dfs = []
for category_code in tqdm(genre_list):
    df = scraping_jcode(category_code,API_KEY,JCODE)
    dfs.append(df)
result = pd.concat(dfs)

100%|██████████| 181/181 [03:36<00:00,  1.19s/it]


In [42]:
result.to_csv(f"Z:\\lab\\yahoo_poi\\origin\\{JCODE}.csv")

OSError: Cannot save file into a non-existent directory: 'Z:\lab\yahoo_poi\origin'

In [81]:
poi = result.copy()

In [83]:
PATH_TO_NON_OFFICE = r"Z:\lab\yahoo_poi\Yahoo POIマスタ - 非就業地.csv"
PATH_TO_OFFICE = r"Z:\lab\yahoo_poi\Yahoo POIマスタ - 就業地.csv"

In [109]:
non_office = pd.read_csv(PATH_TO_NON_OFFICE,dtype=str)
office = pd.read_csv(PATH_TO_OFFICE,dtype=str)
non_office_4 = non_office.copy().drop(columns=["業種コード3","業種名3"]).drop_duplicates()
office_4 = office.copy().drop(columns=["業種コード3","業種名3"]).drop_duplicates()

In [112]:
result_4 = poi[poi['GenreCode'].str.len() == 4]
result_4_office = result_4.merge(office_4,left_on="GenreCode",right_on="業種コード2",how="left")
result_4_added = result_4_office.merge(non_office_4,left_on="GenreCode",right_on="業種コード2",how="left")

result_7 = poi[poi['GenreCode'].str.len() == 7]
result_7_office = result_7.merge(office,left_on="GenreCode",right_on="業種コード3",how="left")
result_7_added = result_7_office.merge(non_office,left_on="GenreCode",right_on="業種コード3",how="left")

In [122]:
poi_added = pd.concat([result_7_added,result_4_added]).filter(items=["Name","BuildingFloor","BuildingArea","X","Y","GenreCode","産業大分類","モデル用定義"]).set_axis(["Name","Floor","Area","Longitude","Latitude","GenreCode","Industry","Activity"],axis=1)

In [124]:
poi_added.to_csv(f"Z:\\lab\\yahoo_poi\\modified\\{JCODE}.csv",index=False)

In [127]:
poi_added

,Name,Floor,Area,Longitude,Latitude,GenreCode,Industry,Activity
0,割烹三寿司,2,127,124.15341151,24.34180227,0101012,M 宿泊業，飲食サービス業,"食事, 仕事のつきあい"
1,こっかーら八重山膳符,2,135,124.16836665,24.352812,0101033,M 宿泊業，飲食サービス業,"食事, 仕事のつきあい"
2,HANARE/フサキビーチリゾート ホテル&ヴィラズ,NaN,NaN,124.114308,24.3700527,0101001,M 宿泊業，飲食サービス業,"食事, 仕事のつきあい"
3,有限会社舟蔵の里,2,111,124.13026591,24.35651634,0101034,M 宿泊業，飲食サービス業,"食事, 仕事のつきあい"
4,八重山/ANAインターコンチネンタル石垣リゾート,12,2803,124.189280,24.3360226,0101001,M 宿泊業，飲食サービス業,"食事, 仕事のつきあい"
...,...,...,...,...,...,...,...,...
234,カレーハウスCoCo壱番屋 沖縄・石垣店,2,580,124.17926536,24.33817635,0107,M 宿泊業，飲食サービス業,"食事, 仕事のつきあい"
235,ミスタードーナツ石垣ショップ,2,145,124.16662191,24.34202093,0115,M 宿泊業，飲食サービス業,"食事, 仕事のつきあい"
236,島のごちそう てぃんがーら,2,111,124.16998283,24.33765493,0110,M 宿泊業，飲食サービス業,"食事, 仕事のつきあい"
237,ロッジビストロ 島のごちそう てぃんがーら 石垣島,2,111,124.16999091,24.33766778,0110,M 宿泊業，飲食サービス業,"食事, 仕事のつきあい"
